## Bayes Classifier

In [62]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from scipy.io import loadmat
import numpy as np
from sklearn.metrics import accuracy_score
from bayes_classifier import *


## Read and load data

In [2]:
def GetAllData():
    data_all = {}
    # data.mat
    # 200 subjects
    # 3 faces per subject
    # size: 24 x 21
    print('-'*10,'Data')
    data = loadmat('data.mat')
    data = data['face']
    print(data.shape)
    # LAbel each 200 classes with their index and set as label
    labels_data = []
    for i in range(data.shape[2]):
        labels_data.append('lbl'+str(i+1))

    data = np.moveaxis(data, -1, 0)
    print(data.shape)
    # Separate 3 faces for the 200 subjects
    data = data.reshape(200,3,(24*21))
    print(data.shape)
    data_all['data'] = (data,labels_data)
    print('-'*10,'Data')

    print('-'*10,'Pose')
    # 68 subjects
    # 13 images per subject (13 different poses)
    # size: 48 x 40
    pose = loadmat('pose.mat')
    pose = pose['pose']
    labels_pose = []
    for i in range(pose.shape[3]):
        labels_pose.append('lbl'+str(i+1))
    print(pose.shape)
    pose = np.moveaxis(np.moveaxis(pose,-1,0),-1,1)
    print(pose.shape)
    data_all['pose'] = (pose,labels_pose)
    print('-'*10,'Pose')

    print('-'*10,'Illum')
    # 68 subjects
    # 21 images per subject (21 different illuminations)
    # size: 48x40
    illum = loadmat('illumination.mat')
    illum = illum['illum']
    labels_illum = []
    print(illum.shape)
    for i in range(illum.shape[2]):
        labels_illum.append('lbl'+str(i+1))
    illum = np.moveaxis(np.moveaxis(illum,-1,0),-1,1)
    print(illum.shape)
    data_all['illum'] = (illum,labels_illum)
    print('-'*10,'Illum')

    return data_all


data_all = GetAllData()


---------- Data
(24, 21, 600)
(600, 24, 21)
(200, 3, 504)
---------- Data
---------- Pose
(48, 40, 13, 68)
(68, 13, 48, 40)
---------- Pose
---------- Illum
(1920, 21, 68)
(68, 21, 1920)
---------- Illum


## Generate random train and test samples

In [3]:
x_train = {}
y_train = {}
#  [data,pose,illum]
n = [15,15,30]
m = [2,7,16]
#
c=0
train_test_all = {}  # Format-> Dataset: (TrainX, TrainY, TestX, TestY)
# Can test the effect of expressions, illumination variations. 
# Here we test the effect of illumination for data.mat
for k in data_all:
    train_datax = data_all[k][0][:,:m[c],:]
    train_datay = data_all[k][1]
    samples = range(data_all[k][0].shape[0])
    # ndarray generated random samples for test data
    rand_indexs = np.array(np.random.choice(samples, n[c], replace = False))
    test_datax = data_all[k][0][rand_indexs,m[c],:]
    test_datay = np.array(data_all[k][1])[rand_indexs]
    train_test_all[k] = (train_datax,train_datay,test_datax,test_datay)
    c+=1
    pass

train_test_all

{'data': (array([[[-1.66302198, -1.8737163 , -1.8737163 , ..., -1.8737163 ,
           -1.8737163 , -1.8737163 ],
          [-1.65747701, -1.87838629, -1.87838629, ..., -1.87838629,
           -1.87838629, -1.87838629]],
  
         [[-1.65700523, -1.87237275, -1.87237275, ..., -1.87237275,
           -1.87237275, -1.87237275],
          [-1.66536705, -1.87703956, -1.87703956, ..., -1.87703956,
           -1.87703956, -1.87703956]],
  
         [[-1.64779442, -1.84646159, -1.84646159, ..., -1.84646159,
           -1.84646159, -1.84646159],
          [-1.62646364, -1.83607769, -1.83607769, ..., -1.83607769,
           -1.83607769, -1.83607769]],
  
         ...,
  
         [[-1.6099227 , -1.83121791, -1.83121791, ..., -1.83121791,
           -1.83121791, -1.83121791],
          [-1.58085847, -1.79983681, -1.79983681, ..., -1.79983681,
           -1.79983681, -1.79983681]],
  
         [[-1.44454051, -1.75942672, -1.75942672, ..., -1.75942672,
           -1.75942672, -1.75942672],
     

## Run without dimensionality reduction:

>> Data.mat

In [4]:
# Add noise = 0.1 to avoid singularity whne calculating covariance.
classifier = BayesClassifier(0.1)
predicted_data1,predicted_data2 = classifier.classify(train_test_all['data'][0],train_test_all['data'][2])
print(accuracy_score(predicted_data1,train_test_all['data'][3]))


0.0


In [14]:
print(train_test_all['data'][3])
print(predicted_data1)

['lbl185' 'lbl178' 'lbl119' 'lbl109' 'lbl113' 'lbl41' 'lbl100' 'lbl93'
 'lbl88' 'lbl78' 'lbl102' 'lbl92' 'lbl162' 'lbl112' 'lbl166']
[184, 177, 118, 52, 53, 40, 99, 92, 192, 65, 152, 59, 161, 111, 165]


>> Pose

In [5]:
# Add noise = 0.1 to avoid singularity whne calculating covariance.
classifier = BayesClassifier(0.1)
train_x = train_test_all['pose'][0]
test_x = train_test_all['pose'][2]
train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2]*train_x.shape[3])
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2])
predicted_pose1,predicted_pose2 = classifier.classify(train_x,test_x)
print(accuracy_score(predicted_pose2,train_test_all['pose'][3]))

0.0


In [15]:
print(train_test_all['pose'][3])
print(predicted_pose2)

['lbl14' 'lbl30' 'lbl8' 'lbl11' 'lbl21' 'lbl15' 'lbl41' 'lbl45' 'lbl31'
 'lbl67' 'lbl20' 'lbl68' 'lbl10' 'lbl56' 'lbl4']
['lbl13', 'lbl29', 'lbl7', 'lbl10', 'lbl20', 'lbl14', 'lbl40', 'lbl65', 'lbl30', 'lbl66', 'lbl19', 'lbl58', 'lbl9', 'lbl55', 'lbl3']


>> Illumination

In [6]:
# Add noise = 0.1 to avoid singularity whne calculating covariance.
classifier = BayesClassifier(0.1)
predicted_illum1,predicted_illum2 = classifier.classify(train_test_all['illum'][0],train_test_all['illum'][2])
print(accuracy_score(predicted_illum1,train_test_all['illum'][3]))

0.0


In [16]:
print(train_test_all['illum'][3])
print(predicted_illum2)

['lbl14' 'lbl44' 'lbl22' 'lbl28' 'lbl32' 'lbl61' 'lbl30' 'lbl39' 'lbl66'
 'lbl57' 'lbl8' 'lbl34' 'lbl52' 'lbl6' 'lbl27' 'lbl51' 'lbl10' 'lbl46'
 'lbl17' 'lbl31' 'lbl38' 'lbl67' 'lbl49' 'lbl18' 'lbl12' 'lbl58' 'lbl1'
 'lbl9' 'lbl65' 'lbl5']
['lbl13', 'lbl43', 'lbl21', 'lbl27', 'lbl31', 'lbl60', 'lbl29', 'lbl38', 'lbl65', 'lbl56', 'lbl7', 'lbl33', 'lbl51', 'lbl5', 'lbl26', 'lbl50', 'lbl9', 'lbl45', 'lbl16', 'lbl30', 'lbl37', 'lbl66', 'lbl48', 'lbl17', 'lbl11', 'lbl57', 'lbl0', 'lbl8', 'lbl64', 'lbl4']


## Principle Component Analysis

In [64]:
from pca import *

>> Data.mat

In [66]:
# Try with different number of components
num_comp = [10,30,50,70,85]
train_x = train_test_all['data'][0]
test_x = train_test_all['data'][2]
# Resize to 2D:
train_x = train_x.reshape(train_x.shape[0]*train_x.shape[1],train_x.shape[2])
for num in num_comp:
    pca = PCA(num)
    pca.fit(train_x)
    train_x_dash = pca.transform(train_x)
    test_x_dash = pca.transform(test_x)
    print(train_x_dash.shape)
    print(test_x_dash.shape)
    train_x_dash = train_x_dash.reshape(200,2,num)

    classifier = BayesClassifier(0.75)
    predicted_data_pca1,predicted_data_pca2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_data_pca2,train_test_all['data'][3]))
    pass


(400, 10)
(15, 10)
0.26666666666666666
(400, 30)
(15, 30)
0.4666666666666667
(400, 50)
(15, 50)
0.5333333333333333
(400, 70)
(15, 70)
0.6
(400, 85)
(15, 85)
0.6


In [17]:
print(predicted_data_pca2)
print(train_test_all['data'][3])

['lbl184', 'lbl177', 'lbl118', 'lbl52', 'lbl53', 'lbl40', 'lbl99', 'lbl92', 'lbl192', 'lbl65', 'lbl152', 'lbl59', 'lbl161', 'lbl111', 'lbl165']
['lbl185' 'lbl178' 'lbl119' 'lbl109' 'lbl113' 'lbl41' 'lbl100' 'lbl93'
 'lbl88' 'lbl78' 'lbl102' 'lbl92' 'lbl162' 'lbl112' 'lbl166']


>> Pose

In [67]:
# Try with different number of components
num_comp = [10,30,50,70,85]
train_x = train_test_all['pose'][0]
test_x = train_test_all['pose'][2]
# Resize to 2D:
train_x = train_x.reshape(train_x.shape[0]*train_x.shape[1],train_x.shape[2]*train_x.shape[3])
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2])

for num in num_comp:
    pca = PCA(num)
    pca.fit(train_x)
    train_x_dash = pca.transform(train_x)
    test_x_dash = pca.transform(test_x)
    print(train_x_dash.shape)
    print(test_x_dash.shape)
    train_x_dash = train_x_dash.reshape(68,7,num)

    classifier = BayesClassifier(0.70)
    predicted_pose_pca1,predicted_pose_pca2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_pose_pca2,train_test_all['pose'][3]))
    pass

(476, 10)
(15, 10)
0.4
(476, 30)
(15, 30)
0.8
(476, 50)
(15, 50)
0.9333333333333333
(476, 70)
(15, 70)
0.9333333333333333
(476, 85)
(15, 85)
0.9333333333333333


>> Illum

In [69]:
# Try with different number of components
num_comp = [10,30,50,70,85]
train_x = train_test_all['illum'][0]
test_x = train_test_all['illum'][2]
# Resize to 2D:
train_x = train_x.reshape(train_x.shape[0]*train_x.shape[1],train_x.shape[2])
for num in num_comp:
    pca = PCA(num)
    pca.fit(train_x)
    train_x_dash = pca.transform(train_x)
    test_x_dash = pca.transform(test_x)
    # print(train_x_dash.shape)
    # print(test_x_dash.shape)
    train_x_dash = train_x_dash.reshape(68,16,num)

    classifier = BayesClassifier(0.15)
    predicted_illum_pca1,predicted_illum_pca2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_illum_pca2,train_test_all['illum'][3]))
    pass


0.5
1.0
1.0
1.0
1.0


## Multiple Discriminant Analysis (MDA)

In [20]:
from mda import *

>> Data.mat

In [71]:
train_x = train_test_all['data'][0]
test_x = train_test_all['data'][2]
num_newdim = [10,30,50,70,85]
for n_dim in num_newdim:
    mda_data = MDA(0.9,train_x,test_x,n_dim)
    mda_data.get_matA()
    train_x_dash = mda_data.transform(train_x)
    test_x_dash = mda_data.transform(test_x)

    # Add noise = 0.1 to avoid singularity whne calculating covariance.
    classifier = BayesClassifier(0.95)
    predicted_data_mda1,predicted_data_mda2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_data_mda2,train_test_all['data'][3]))


0.26666666666666666
0.4666666666666667
0.5333333333333333
0.6
0.6


>> Pose

In [72]:
train_x = train_test_all['pose'][0]
test_x = train_test_all['pose'][2]
train_x = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2]*train_x.shape[3])
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2])
num_newdim = [10,30,50,70,85]
for n_dim in num_newdim:
    mda_pose = MDA(0.5,train_x,test_x,n_dim)
    mda_pose.get_matA()
    train_x_dash = mda_pose.transform(train_x)
    test_x_dash = mda_pose.transform(test_x)
    # Add noise = 0.8 to avoid singularity whne calculating covariance.
    classifier = BayesClassifier(0.8)
    predicted_pose_mda1,predicted_pose_mda2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_pose_mda2,train_test_all['pose'][3]))

0.7333333333333333
0.9333333333333333
0.9333333333333333
0.8666666666666667
0.8


>> Illumination

In [73]:
train_x = train_test_all['illum'][0]
test_x = train_test_all['illum'][2]

num_newdim = [10,30,50,70,85]
for n_dim in num_newdim:
    mda_illum = MDA(0.1,train_x,test_x,n_dim)
    mda_illum.get_matA()
    train_x_dash = mda_illum.transform(train_x)
    test_x_dash = mda_illum.transform(test_x)

    classifier = BayesClassifier(0.75)
    predicted_illum_mda1,predicted_illum_mda2 = classifier.classify(train_x_dash,test_x_dash)
    print(accuracy_score(predicted_illum_mda2,train_test_all['illum'][3]))

0.6666666666666666
1.0
1.0
1.0
1.0
